# Modern NLP in Python
### _- Or -_
## What you can learn about food by analyzing a million Yelp reviews

#### Before we get started...
__whois?__
- Patrick Harrison
- Lead Data Scientist @ S&P Global Market Intelligence - _**we are hiring**_
- University of Virginia &mdash; Systems Engineering
- patrick@skipgram.io / @skipgram

__Join Charlottesville Data Science!__
- On Meetup.com ... http://www.meetup.com/CharlottesvilleDataScience
- On Slack ... __https://cville.typeform.com/to/UEzMVh__
  - _link invites you to join the Cville team on Slack. Join Cville, then join channel __#datascience__._
  
_Note: I presented this notebook as a tutorial during the [PyData DC 2016 conference](http://pydata.org/dc2016/schedule/presentation/11/). To view the video of the presentation on YouTube, see [here](https://www.youtube.com/watch?v=6zm9NC9uRkk)._

## Our Trail Map
This tutorial features an end-to-end data science & natural language processing pipeline, starting with **raw data** and running through **preparing**, **modeling**, **visualizing**, and **analyzing** the data. We'll touch on the following points:
1. A tour of the dataset
1. Introduction to text processing with spaCy
1. Automatic phrase modeling
1. Topic modeling with LDA
1. Visualizing topic models with pyLDAvis
1. Word vector models with word2vec
1. Visualizing word2vec with t-SNE

...and we might even learn a thing or two about Python along the way.

Let's get started!

## The Yelp Dataset
[**The Yelp Dataset**](https://www.yelp.com/dataset_challenge/) is a dataset published by the business review service [Yelp](http://yelp.com) for academic research and educational purposes. I really like the Yelp dataset as a subject for machine learning and natural language processing demos, because it's big (but not so big that you need your own data center to process it), well-connected, and anyone can relate to it &mdash; it's largely about food, after all!

**Note:** If you'd like to execute this notebook interactively on your local machine, you'll need to download your own copy of the Yelp dataset. If you're reviewing a static copy of the notebook online, you can skip this step. Here's how to get the dataset:
1. Please visit the Yelp dataset webpage [here](https://www.yelp.com/dataset_challenge/)
1. Click "Get the Data"
1. Please review, agree to, and respect Yelp's terms of use!
1. The dataset downloads as a compressed .tgz file; uncompress it
1. Place the uncompressed dataset files (*yelp_academic_dataset_business.json*, etc.) in a directory named *yelp_dataset_challenge_academic_dataset*
1. Place the *yelp_dataset_challenge_academic_dataset* within the *data* directory in the *Modern NLP in Python* project folder

That's it! You're ready to go.

The current iteration of the Yelp dataset (as of this demo) consists of the following data:
- __1M__ users
- __144K__ businesses
- __4.1M__ user reviews

When focusing on restaurants alone, there are approximately __49K__ restaurants with approximately __2.6M__ user reviews written about them.

The data is provided in a handful of files in _.json_ format. We'll be using the following files for our demo:
- __yelp\_academic\_dataset\_business.json__ &mdash; _the records for individual businesses_
- __yelp\_academic\_dataset\_review.json__ &mdash; _the records for reviews users wrote about businesses_

The files are text files (UTF-8) with one _json object_ per line, each one corresponding to an individual data record. Let's take a look at a few examples.

In [1]:
import os
import codecs

data_directory = os.path.join('..', 'data',
                              'yelp_dataset_challenge_academic_dataset')

businesses_filepath = os.path.join(data_directory,
                                   'yelp_academic_dataset_business.json')

with codecs.open(businesses_filepath, encoding='utf_8') as f:
    first_business_record = f.readline() 

print(first_business_record)

{"business_id":"0DI8Dt2PJp07XkVvIElIcQ","name":"Innovative Vapors","neighborhood":"","address":"227 E Baseline Rd, Ste J2","city":"Tempe","state":"AZ","postal_code":"85283","latitude":33.3782141,"longitude":-111.936102,"stars":4.5,"review_count":17,"is_open":0,"attributes":["BikeParking: True","BusinessAcceptsBitcoin: False","BusinessAcceptsCreditCards: True","BusinessParking: {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}","DogsAllowed: False","RestaurantsPriceRange2: 2","WheelchairAccessible: True"],"categories":["Tobacco Shops","Nightlife","Vape Shops","Shopping"],"hours":["Monday 11:0-21:0","Tuesday 11:0-21:0","Wednesday 11:0-21:0","Thursday 11:0-21:0","Friday 11:0-22:0","Saturday 10:0-22:0","Sunday 11:0-18:0"],"type":"business"}



The business records consist of _key, value_ pairs containing information about the particular business. A few attributes we'll be interested in for this demo include:
- __business\_id__ &mdash; _unique identifier for businesses_
- __categories__ &mdash; _an array containing relevant category values of businesses_

The _categories_ attribute is of special interest. This demo will focus on restaurants, which are indicated by the presence of the _Restaurant_ tag in the _categories_ array. In addition, the _categories_ array may contain more detailed information about restaurants, such as the type of food they serve.

The review records are stored in a similar manner &mdash; _key, value_ pairs containing information about the reviews.

In [2]:
review_json_filepath = os.path.join(data_directory,
                                    'yelp_academic_dataset_review.json')

with codecs.open(review_json_filepath, encoding='utf_8') as f:
    first_review_record = f.readline()
    
print(first_review_record)

{"review_id":"NxL8SIC5yqOdnlXCg18IBg","user_id":"KpkOkG6RIf4Ra25Lhhxf1A","business_id":"2aFiy99vNLklCx3T_tGS9A","stars":5,"date":"2011-10-10","text":"If you enjoy service by someone who is as competent as he is personable, I would recommend Corey Kaplan highly. The time he has spent here has been very productive and working with him educational and enjoyable. I hope not to need him again (though this is highly unlikely) but knowing he is there if I do is very nice. By the way, I'm not from El Centro, CA. but Scottsdale, AZ.","useful":0,"funny":0,"cool":0,"type":"review"}



A few attributes of note on the review records:
- __business\_id__ &mdash; _indicates which business the review is about_
- __text__ &mdash; _the natural language text the user wrote_

The _text_ attribute will be our focus today!

_json_ is a handy file format for data interchange, but it's typically not the most usable for any sort of modeling work. Let's do a bit more data preparation to get our data in a more usable format. Our next code block will do the following:
1. Read in each business record and convert it to a Python `dict`
2. Filter out business records that aren't about restaurants (i.e., not in the "Restaurant" category)
3. Create a `frozenset` of the business IDs for restaurants, which we'll use in the next step

In [3]:
import json

restaurant_ids = set()

# open the businesses file
with codecs.open(businesses_filepath, encoding='utf_8') as f:
    
    # iterate through each line (json record) in the file
    for business_json in f:
        
        # convert the json record to a Python dict
        business = json.loads(business_json)
        # if this business is not a restaurant, skip to the next one
        if (business['categories'] is not None and
                'Restaurants' not in business['categories']):
            continue
            
        # add the restaurant business id to our restaurant_ids set
        restaurant_ids.add(business['business_id'])

# turn restaurant_ids into a frozenset, as we don't need to change it anymore
restaurant_ids = frozenset(restaurant_ids)

# print the number of unique restaurant ids in the dataset
print('{:,}'.format(len(restaurant_ids)), 'restaurants in the dataset.')

48,810 restaurants in the dataset.


Next, we will create a new file that contains only the text from reviews about restaurants, with one review per line in the file.

In [4]:
intermediate_directory = os.path.join('..', 'intermediate')

review_txt_filepath = os.path.join(intermediate_directory,
                                   'review_text_all.txt')

In [5]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 0:
    
    review_count = 0

    # create & open a new file in write mode
    with codecs.open(review_txt_filepath, 'w',
                     encoding='utf_8') as review_txt_file:

        # open the existing review json file
        with codecs.open(review_json_filepath,
                         encoding='utf_8') as review_json_file:

            # loop through all reviews in the existing file and convert to dict
            for review_json in review_json_file:
                review = json.loads(review_json)

                # if this review is not about a restaurant,
                # skip to the next one
                if review['business_id'] not in restaurant_ids:
                    continue

                # write the restaurant review as a line in the new file
                # escape newline characters in the original review text
                review_txt_file.write(review['text'].replace(
                    '\n', '\\n') + '\n')
                review_count += 1

    print('''Text from {:,} restaurant reviews
              written to the new txt file.'''.format(review_count))
    
else:
    
    with codecs.open(review_txt_filepath, encoding='utf_8') as review_txt_file:
        for review_count, line in enumerate(review_txt_file):
            pass
        
    print('Text from {:,} restaurant reviews in the txt file.'.format(
        review_count + 1))

Text from 2,580,713 restaurant reviews in the txt file.
CPU times: user 1min 12s, sys: 744 ms, total: 1min 12s
Wall time: 1min 12s


## spaCy &mdash; Industrial-Strength NLP in Python|

![spaCy](https://s3.amazonaws.com/skipgram-images/spaCy.png)

[**spaCy**](https://spacy.io) is an industrial-strength natural language processing (_NLP_) library for Python. spaCy's goal is to take recent advancements in natural language processing out of research papers and put them in the hands of users to build production software.

spaCy handles many tasks commonly associated with building an end-to-end natural language processing pipeline:
- Tokenization
- Text normalization, such as lowercasing, stemming/lemmatization
- Part-of-speech tagging
- Syntactic dependency parsing
- Sentence boundary detection
- Named entity recognition and annotation

In the "batteries included" Python tradition, spaCy contains built-in data and models which you can use out-of-the-box for processing general-purpose English language text:
- Large English vocabulary, including stopword lists
- Token "probabilities"
- Word vectors

spaCy is written in optimized Cython, which means it's _fast_. According to a few independent sources, it's the fastest syntactic parser available in any language. Key pieces of the spaCy parsing pipeline are written in pure C, enabling efficient multithreading (i.e., spaCy can release the _GIL_).

In [6]:
# install English model
#!python -m spacy.en.download

In [7]:
import spacy
import pandas as pd
import itertools as it

nlp = spacy.load('en')

Let's grab a sample review to play with.

In [8]:
with codecs.open(review_txt_filepath, encoding='utf_8') as f:
    sample_review = list(it.islice(f, 8, 9))[0]
    sample_review = sample_review.replace('\\n', '\n')
        
print(sample_review)

Was boozin and cruisin with my canine comrade. Went all the way out to Hudson via Brecksville/82. Looking for anything to Yelp. Nothing. Not even in the strip mall I used to hit in Hudson on my dry cleaning route back in the day. There was a deli, but it opened at fucking 11! Who even wants to go to a deli after 11? Weak. Breezed through Peninsula. Winking Lizard? Bah. Ended up backtracking through Brecksville and I remembered an old friend. London Pickle Works! Was it still there some four years after I stopped in while house shopping? Yup. Incidentally, investing in a condo down there probably would have made more sense than buying a house in Lakewood. Live and learn.

Cozied up to the bar and ordered a Killians, plus cheesesteak to go(for pooch). Like the Unicorn, food was amazingly done before I could chug my brew. I could get used to this kind of service. The bar was all old men. Matt, I don't know how you do it. They were watching Fox news for chrissakes!! Total was eight dollars

Hand the review text to spaCy, and be prepared to wait...

In [9]:
%%time
parsed_review = nlp(sample_review)

CPU times: user 32 ms, sys: 0 ns, total: 32 ms
Wall time: 32.4 ms


...1/20th of a second or so. Let's take a look at what we got during that time...

In [10]:
print(parsed_review)

Was boozin and cruisin with my canine comrade. Went all the way out to Hudson via Brecksville/82. Looking for anything to Yelp. Nothing. Not even in the strip mall I used to hit in Hudson on my dry cleaning route back in the day. There was a deli, but it opened at fucking 11! Who even wants to go to a deli after 11? Weak. Breezed through Peninsula. Winking Lizard? Bah. Ended up backtracking through Brecksville and I remembered an old friend. London Pickle Works! Was it still there some four years after I stopped in while house shopping? Yup. Incidentally, investing in a condo down there probably would have made more sense than buying a house in Lakewood. Live and learn.

Cozied up to the bar and ordered a Killians, plus cheesesteak to go(for pooch). Like the Unicorn, food was amazingly done before I could chug my brew. I could get used to this kind of service. The bar was all old men. Matt, I don't know how you do it. They were watching Fox news for chrissakes!! Total was eight dollars

Looks the same! What happened under the hood?

What about sentence detection and segmentation?

In [11]:
for num, sentence in enumerate(parsed_review.sents):
    print('Sentence {}:'.format(num + 1))
    print(sentence)
    print('')

Sentence 1:
Was boozin and cruisin with my canine comrade.

Sentence 2:
Went all the way out to Hudson via Brecksville/82.

Sentence 3:
Looking for anything to Yelp.

Sentence 4:
Nothing.

Sentence 5:
Not even in the strip mall I used to hit in Hudson on my dry cleaning route back in the day.

Sentence 6:
There was a deli, but it opened at fucking 11!

Sentence 7:
Who even wants to go to a deli after 11?

Sentence 8:
Weak.

Sentence 9:
Breezed through Peninsula.

Sentence 10:
Winking Lizard?

Sentence 11:
Bah.

Sentence 12:
Ended up backtracking through Brecksville and I remembered an old friend.

Sentence 13:
London Pickle Works!

Sentence 14:
Was it still there some four years after I stopped in while house shopping?

Sentence 15:
Yup.

Sentence 16:
Incidentally, investing in a condo down there probably would have made more sense than buying a house in Lakewood.

Sentence 17:
Live and learn.



Sentence 18:
Cozied up to the bar and ordered a Killians, plus cheesesteak to go(for pooch

What about named entity detection?

In [12]:
for num, entity in enumerate(parsed_review.ents):
    print('Entity {}:'.format(num + 1), entity, '-', entity.label_)
    print('')

Entity 1: Hudson - PERSON

Entity 2: Hudson - PERSON

Entity 3: the day - DATE

Entity 4: 11 - CARDINAL

Entity 5: Peninsula - PERSON

Entity 6: Brecksville - PERSON

Entity 7: London Pickle Works - ORG

Entity 8: some four years - DATE

Entity 9: Lakewood - GPE

Entity 10: Killians - NORP

Entity 11: Matt - PERSON

Entity 12: Fox - ORG

Entity 13: eight - CARDINAL

Entity 14: three - CARDINAL

Entity 15: one - CARDINAL

Entity 16: NES. - ORG

Entity 17: eight - CARDINAL

Entity 18: four more years - DATE



What about part of speech tagging?

In [13]:
token_text = [token.orth_ for token in parsed_review]
token_pos = [token.pos_ for token in parsed_review]

pd.DataFrame(list(zip(token_text, token_pos)),
             columns=['token_text', 'part_of_speech'])

,token_text,part_of_speech
0,Was,VERB
1,boozin,NOUN
2,and,CONJ
3,cruisin,NOUN
4,with,ADP
5,my,ADJ
6,canine,ADJ
7,comrade,NOUN
8,.,PUNCT
9,Went,VERB


What about text normalization, like stemming/lemmatization and shape analysis?

In [14]:
token_lemma = [token.lemma_ for token in parsed_review]
token_shape = [token.shape_ for token in parsed_review]

pd.DataFrame(list(zip(token_text, token_lemma, token_shape)),
             columns=['token_text', 'token_lemma', 'token_shape'])

,token_text,token_lemma,token_shape
0,Was,be,Xxx
1,boozin,boozin,xxxx
2,and,and,xxx
3,cruisin,cruisin,xxxx
4,with,with,xxxx
5,my,my,xx
6,canine,canine,xxxx
7,comrade,comrade,xxxx
8,.,.,.
9,Went,go,Xxxx


What about token-level entity analysis?

In [15]:
token_entity_type = [token.ent_type_ for token in parsed_review]
token_entity_iob = [token.ent_iob_ for token in parsed_review]

pd.DataFrame(list(zip(token_text, token_entity_type, token_entity_iob)),
             columns=['token_text', 'entity_type', 'inside_outside_begin'])

,token_text,entity_type,inside_outside_begin
0,Was,,O
1,boozin,,O
2,and,,O
3,cruisin,,O
4,with,,O
5,my,,O
6,canine,,O
7,comrade,,O
8,.,,O
9,Went,,O


What about a variety of other token-level attributes, such as the relative frequency of tokens, and whether or not a token matches any of these categories?
- stopword
- punctuation
- whitespace
- represents a number
- whether or not the token is included in spaCy's default vocabulary?

In [16]:
token_attributes = [(token.orth_,
                     token.prob,
                     token.is_stop,
                     token.is_punct,
                     token.is_space,
                     token.like_num,
                     token.is_oov)
                    for token in parsed_review]

df = pd.DataFrame(token_attributes,
                  columns=['text',
                           'log_probability',
                           'stop?',
                           'punctuation?',
                           'whitespace?',
                           'number?',
                           'out of vocab.?'])

df.loc[:, 'stop?':'out of vocab.?'] = (df.loc[:, 'stop?':'out of vocab.?']
                                       .applymap(lambda x: 'Yes'
                                                 if x else ''))
                                               
df

,text,log_probability,stop?,punctuation?,whitespace?,number?,out of vocab.?
0,Was,-9.648565,Yes,,,,
1,boozin,-17.988129,,,,,
2,and,-4.113108,Yes,,,,
3,cruisin,-16.285810,,,,,
4,with,-5.243250,Yes,,,,
5,my,-5.491643,Yes,,,,
6,canine,-14.164199,,,,,
7,comrade,-13.486403,,,,,
8,.,-3.067898,,Yes,,,
9,Went,-11.492407,,,,,


If the text you'd like to process is general-purpose English language text (i.e., not domain-specific, like medical literature), spaCy is ready to use out-of-the-box.

I think it will eventually become a core part of the Python data science ecosystem &mdash; it will do for natural language computing what other great libraries have done for numerical computing.

## Phrase Modeling

_Phrase modeling_ is another approach to learning combinations of tokens that together represent meaningful multi-word concepts. We can develop phrase models by looping over the the words in our reviews and looking for words that _co-occur_ (i.e., appear one after another) together much more frequently than you would expect them to by random chance. The formula our phrase models will use to determine whether two tokens $A$ and $B$ constitute a phrase is:

$$\frac{count(A\ B) - count_{min}}{count(A) * count(B)} * N > threshold$$

...where:
* $count(A)$ is the number of times token $A$ appears in the corpus
* $count(B)$ is the number of times token $B$ appears in the corpus
* $count(A\ B)$ is the number of times the tokens $A\ B$ appear in the corpus *in order*
* $N$ is the total size of the corpus vocabulary
* $count_{min}$ is a user-defined parameter to ensure that accepted phrases occur a minimum number of times
* $threshold$ is a user-defined parameter to control how strong of a relationship between two tokens the model requires before accepting them as a phrase

Once our phrase model has been trained on our corpus, we can apply it to new text. When our model encounters two tokens in new text that identifies as a phrase, it will merge the two into a single new token.

Phrase modeling is superficially similar to named entity detection in that you would expect named entities to become phrases in the model (so _new york_ would become *new\_york*). But you would also expect multi-word expressions that represent common concepts, but aren't specifically named entities (such as _happy hour_) to also become phrases in the model.

We turn to the indispensible [**gensim**](https://radimrehurek.com/gensim/index.html) library to help us with phrase modeling &mdash; the [**Phrases**](https://radimrehurek.com/gensim/models/phrases.html) class in particular.

In [17]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.models.word2vec import LineSentence

As we're performing phrase modeling, we'll be doing some iterative data transformation at the same time. Our roadmap for data preparation includes:

1. Segment text of complete reviews into sentences & normalize text
1. First-order phrase modeling $\rightarrow$ _apply first-order phrase model to transform sentences_
1. Second-order phrase modeling $\rightarrow$ _apply second-order phrase model to transform sentences_
1. Apply text normalization and second-order phrase model to text of complete reviews

We'll use this transformed data as the input for some higher-level modeling approaches in the following sections.

First, let's define a few helper functions that we'll use for text normalization. In particular, the `lemmatized_sentence_corpus` generator function will use spaCy to:
- Iterate over the 1M reviews in the `review_txt_all.txt` we created before
- Segment the reviews into individual sentences
- Remove punctuation and excess whitespace
- Lemmatize the text

... and do so efficiently in parallel, thanks to spaCy's `nlp.pipe()` function.

In [18]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield ' '.join([token.lemma_ for token in sent
                            if not punct_space(token)])

In [19]:
unigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'unigram_sentences_all.txt')

Let's use the `lemmatized_sentence_corpus` generator to loop over the original review text, segmenting the reviews into individual sentences and normalizing the text. We'll write this data back out to a new file (`unigram_sentences_all`), with one normalized sentence per line. We'll use this data for learning our phrase models.

In [20]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 0:

    with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus(review_txt_filepath):
            f.write(sentence + '\n')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.15 µs


If your data is organized like our `unigram_sentences_all` file now is &mdash; a large text file with one document/sentence per line &mdash; gensim's [**LineSentence**](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.LineSentence) class provides a convenient iterator for working with other gensim components. It *streams* the documents/sentences from disk, so that you never have to hold the entire corpus in RAM at once. This allows you to scale your modeling pipeline up to potentially very large corpora.

In [21]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

Let's take a look at a few sample sentences in our new, transformed file.

In [22]:
for unigram_sentence in it.islice(unigram_sentences, 210, 220):
    print(' '.join(unigram_sentence))
    print('')

once you enter it ' worth the search

i really like the safe deposit box in the entryway make for some cute picture

food yum

i think it may be hit or miss base on read other review but we choose wisely

here ' what i like spicy tuna roll just enough spice quality fish curry noodle love the curry they use definitely singaporean

big shrimp

ginger chicken with broccoli a light option with clean flavor

service great

our sever recommend his favorite drink and be helpful answer question about the food

this would be a great date spot



Next, we'll learn a phrase model that will link individual words into two-word phrases. We'd expect words that together represent a specific concept, like "`scipy tuna`", to be linked together to form a new, single token: "`scpicy_tuna`".

In [23]:
bigram_model_filepath = os.path.join(intermediate_directory,
                                     'bigram_model_all')

In [24]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 1 == 0:

    bigram_model = Phrases(unigram_sentences)

    bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk
bigram_model = Phraser(Phrases.load(bigram_model_filepath))

CPU times: user 3min 3s, sys: 1.22 s, total: 3min 4s
Wall time: 3min 3s


Now that we have a trained phrase model for word pairs, let's apply it to the review sentences data and explore the results.

In [25]:
bigram_sentences_filepath = os.path.join(intermediate_directory,
                                         'bigram_sentences_all.txt')

In [26]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 0:

    with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for unigram_sentence in unigram_sentences:
            
            bigram_sentence = ' '.join(bigram_model[unigram_sentence])
            
            f.write(bigram_sentence + '\n')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.91 µs


In [27]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [28]:
for bigram_sentence in it.islice(bigram_sentences, 210, 220):
    print(' '.join(bigram_sentence))
    print('')

once you enter it ' worth the search

i really like the safe deposit box in the entryway make for some cute picture

food yum

i think it may be hit or miss base on read other review but we choose_wisely

here ' what i like spicy_tuna roll just enough spice quality fish curry noodle love the curry they use definitely singaporean

big shrimp

ginger chicken with broccoli a light option with clean flavor

service great

our sever recommend his favorite drink and be helpful answer_question about the food

this would be a great date spot



Looks like the phrase modeling worked! We now see two-word phrases, such as "`spicy_tuna`", linked together in the text as a single token. Next, we'll train a _second-order_ phrase model. We'll apply the second-order phrase model on top of the already-transformed data, so that incomplete word combinations like "`spicy_tuna roll`" will become fully joined to "`spicy_tuna_roll`".

In [29]:
trigram_model_filepath = os.path.join(intermediate_directory,
                                      'trigram_model_all')

In [30]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 1 == 0:

    trigram_model = Phrases(bigram_sentences)

    trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phraser(Phrases.load(trigram_model_filepath))

CPU times: user 4min 17s, sys: 1.78 s, total: 4min 19s
Wall time: 4min 17s


We'll apply our trained second-order phrase model to our first-order transformed sentences, write the results out to a new file, and explore a few of the second-order transformed sentences.

In [31]:
trigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'trigram_sentences_all.txt')

In [32]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 0:

    with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for bigram_sentence in bigram_sentences:
            
            trigram_sentence = ' '.join(trigram_model[bigram_sentence])
            
            f.write(trigram_sentence + '\n')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.44 µs


In [33]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [34]:
for trigram_sentence in it.islice(trigram_sentences, 210, 220):
    print(' '.join(trigram_sentence))
    print('')

once you enter it ' worth the search

i really like the safe deposit box in the entryway make for some cute picture

food yum

i think it may be hit or miss base on read other review but we choose_wisely

here ' what i like spicy_tuna_roll just enough spice quality fish curry noodle love the curry they use definitely singaporean

big shrimp

ginger chicken with broccoli a light option with clean flavor

service great

our sever recommend his favorite drink and be helpful answer_question about the food

this would be a great date spot



Looks like the second-order phrase model was successful. We're now seeing three-word phrases, such as "`spicy_tuna_roll`".

The final step of our text preparation process circles back to the complete text of the reviews. We're going to run the complete text of the reviews through a pipeline that applies our text normalization and phrase models.

In addition, we'll remove stopwords at this point. _Stopwords_ are very common words, like _a_, _the_, _and_, and so on, that serve functional roles in natural language, but typically don't contribute to the overall meaning of text. Filtering stopwords is a common procedure that allows higher-level NLP modeling techniques to focus on the words that carry more semantic weight.

Finally, we'll write the transformed text out to a new file, with one review per line.

In [35]:
trigram_reviews_filepath = os.path.join(intermediate_directory,
                                        'trigram_transformed_reviews_all.txt')

In [36]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 0:

    with codecs.open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:
        
        for parsed_review in nlp.pipe(line_review(review_txt_filepath),
                                      batch_size=10000, n_threads=4):
            
            # lemmatize the text, removing punctuation and whitespace
            unigram_review = [token.lemma_ for token in parsed_review
                              if not punct_space(token)]
            
            # apply the first-order and second-order phrase models
            bigram_review = bigram_model[unigram_review]
            trigram_review = trigram_model[bigram_review]
            
            # remove any remaining stopwords
            trigram_review = [term for term in trigram_review
                              if term not in spacy.en.STOPWORDS]
            
            # write the transformed review as a line in the new file
            trigram_review = ' '.join(trigram_review)
            f.write(trigram_review + '\n')

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.91 µs


Let's preview the results. We'll grab one review from the file with the original, untransformed text, grab the same review from the file with the normalized and transformed text, and compare the two.

In [37]:
print('Original:' + '\n')

for review in it.islice(line_review(review_txt_filepath), 11, 12):
    print(review)

print('----' + '\n')
print('Transformed:' + '\n')

with codecs.open(trigram_reviews_filepath, encoding='utf_8') as f:
    for review in it.islice(f, 11, 12):
        print(review)

Original:

3 stars is fair for this pub. 

The food is good and well priced for a bar. 

Tuesday has $5 burgers and chicken sandwiches. 

The pecan pie is good for dessert. 

I got the chicken dinner, it came with a 6 oz chicken, broccoli, steamed potatoes, salad, and roll. It was ok. I'd stick with the burgers though. The salad was fresh. 

Their fries are really good.

----

Transformed:

3 star fair pub food good price bar tuesday $ 5 burger chicken sandwich pecan_pie good dessert chicken dinner come 6_oz chicken broccoli steam potato salad roll ok stick burger salad fresh fries good



You can see that most of the grammatical structure has been scrubbed from the text &mdash; capitalization, articles/conjunctions, punctuation, spacing, etc. However, much of the general semantic *meaning* is still present. Also, multi-word concepts such as "`pecan_pie`" and "`6_oz`" have been joined into single tokens, as expected. The review text is now ready for higher-level modeling. 

## Topic Modeling with Latent Dirichlet Allocation (_LDA_)

*Topic modeling* is family of techniques that can be used to describe and summarize the documents in a corpus according to a set of latent "topics". For this demo, we'll be using [*Latent Dirichlet Allocation*](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf) or LDA, a popular approach to topic modeling.

In many conventional NLP applications, documents are represented a mixture of the individual tokens (words and phrases) they contain. In other words, a document is represented as a *vector* of token counts. There are two layers in this model &mdash; documents and tokens &mdash; and the size or dimensionality of the document vectors is the number of tokens in the corpus vocabulary. This approach has a number of disadvantages:
* Document vectors tend to be large (one dimension for each token $\Rightarrow$ lots of dimensions)
* They also tend to be very sparse. Any given document only contains a small fraction of all tokens in the vocabulary, so most values in the document's token vector are 0.
* The dimensions are fully indepedent from each other &mdash; there's no sense of connection between related tokens, such as _knife_ and _fork_.

LDA injects a third layer into this conceptual model. Documents are represented as a mixture of a pre-defined number of *topics*, and the *topics* are represented as a mixture of the individual tokens in the vocabulary. The number of topics is a model hyperparameter selected by the practitioner. LDA makes a prior assumption that the (document, topic) and (topic, token) mixtures follow [*Dirichlet*](https://en.wikipedia.org/wiki/Dirichlet_distribution) probability distributions. This assumption encourages documents to consist mostly of a handful of topics, and topics to consist mostly of a modest set of the tokens.

![LDA](https://s3.amazonaws.com/skipgram-images/LDA.png)

LDA is fully unsupervised. The topics are "discovered" automatically from the data by trying to maximize the likelihood of observing the documents in your corpus, given the modeling assumptions. They are expected to capture some latent structure and organization within the documents, and often have a meaningful human interpretation for people familiar with the subject material.

We'll again turn to gensim to assist with data preparation and modeling. In particular, gensim offers a high-performance parallelized implementation of LDA with its [**LdaMulticore**](https://radimrehurek.com/gensim/models/ldamulticore.html) class.

In [38]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle

/home/fehiepsi/miniconda3/envs/pydata/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/fehiepsi/miniconda3/envs/pydata/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/fehiepsi/miniconda3/envs/pydata/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/fehiepsi/miniconda3/envs/pydata/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
/home/fehiepsi/miniconda3/envs/pydata/lib/python3.5/site-packages/funcy/decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use in

The first step to creating an LDA model is to learn the full vocabulary of the corpus to be modeled. We'll use gensim's [**Dictionary**](https://radimrehurek.com/gensim/corpora/dictionary.html) class for this.

In [39]:
trigram_dictionary_filepath = os.path.join(intermediate_directory,
                                           'trigram_dict_all.dict')

In [40]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to learn the dictionary yourself.
if 1 == 0:

    trigram_reviews = LineSentence(trigram_reviews_filepath)

    # learn the dictionary by iterating over all of the reviews
    trigram_dictionary = Dictionary(trigram_reviews)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
    trigram_dictionary.compactify()

    trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

CPU times: user 64 ms, sys: 0 ns, total: 64 ms
Wall time: 118 ms


Like many NLP techniques, LDA uses a simplifying assumption known as the [*bag-of-words* model](https://en.wikipedia.org/wiki/Bag-of-words_model). In the bag-of-words model, a document is represented by the counts of distinct terms that occur within it. Additional information, such as word order, is discarded. 

Using the gensim Dictionary we learned to generate a bag-of-words representation for each review. The `trigram_bow_generator` function implements this. We'll save the resulting bag-of-words reviews as a matrix.

In the following code, "bag-of-words" is abbreviated as `bow`.

In [41]:
trigram_bow_filepath = os.path.join(intermediate_directory,
                                    'trigram_bow_corpus_all.mm')

In [42]:
def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """
    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [43]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to build the bag-of-words corpus yourself.
if 1 == 0:

    # generate bag-of-words representations for
    # all reviews and save them as a matrix
    MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_reviews_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

CPU times: user 392 ms, sys: 16 ms, total: 408 ms
Wall time: 505 ms


With the bag-of-words corpus, we're finally ready to learn our topic model from the reviews. We simply need to pass the bag-of-words matrix and Dictionary from our previous steps to `LdaMulticore` as inputs, along with the number of topics the model should learn. For this demo, we're asking for 50 topics.

In [44]:
lda_model_filepath = os.path.join(intermediate_directory, 'lda_model_all')

In [45]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the LDA model yourself.
if 1 == 0:

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=50,
                           id2word=trigram_dictionary,
                           workers=4)
    
    lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

CPU times: user 352 ms, sys: 68 ms, total: 420 ms
Wall time: 647 ms


Our topic model is now trained and ready to use! Since each topic is represented as a mixture of tokens, you can manually inspect which tokens have been grouped together into which topics to try to understand the patterns the model has discovered in the data.

In [46]:
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print('{:20} {}'.format('term', 'frequency') + '\n')

    for term, frequency in lda.show_topic(topic_number, topn=25):
        print('{:20} {:.3f}'.format(term, round(frequency, 3)))

In [47]:
explore_topic(topic_number=0)

term                 frequency

lemonade             0.037
champagne            0.030
wedding              0.029
chris                0.023
cat                  0.019
ipa                  0.016
rise                 0.015
west_side            0.014
bustle               0.013
lake                 0.012
hustle               0.011
vanilla_ice_cream    0.011
dane                 0.011
kool_aid             0.011
r                    0.011
monkey_bread         0.010
l                    0.010
waterfall            0.010
cowboy               0.009
steve                0.008
reception            0.008
scar                 0.008
peep                 0.008
ruth_'s              0.007
pit                  0.007


The first topic has strong associations with words like *taco*, *salsa*, *chip*, *burrito*, and *margarita*, as well as a handful of more general words. You might call this the **Mexican food** topic!

It's possible to go through and inspect each topic in the same way, and try to assign a human-interpretable label that captures the essence of each one. I've given it a shot for all 50 topics below.

In [48]:
topic_names = {0: u'mexican',
               1: u'menu',
               2: u'thai',
               3: u'steak',
               4: u'donuts & appetizers',
               5: u'specials',
               6: u'soup',
               7: u'wings, sports bar',
               8: u'foreign language',
               9: u'las vegas',
               10: u'chicken',
               11: u'aria buffet',
               12: u'noodles',
               13: u'ambience & seating',
               14: u'sushi',
               15: u'arizona',
               16: u'family',
               17: u'price',
               18: u'sweet',
               19: u'waiting',
               20: u'general',
               21: u'tapas',
               22: u'dirty',
               23: u'customer service',
               24: u'restrooms',
               25: u'chinese',
               26: u'gluten free',
               27: u'pizza',
               28: u'seafood',
               29: u'amazing',
               30: u'eat, like, know, want',
               31: u'bars',
               32: u'breakfast',
               33: u'location & time',
               34: u'italian',
               35: u'barbecue',
               36: u'arizona',
               37: u'indian',
               38: u'latin & cajun',
               39: u'burger & fries',
               40: u'vegetarian',
               41: u'lunch buffet',
               42: u'customer service',
               43: u'taco, ice cream',
               44: u'high cuisine',
               45: u'healthy',
               46: u'salad & sandwich',
               47: u'greek',
               48: u'poor experience',
               49: u'wine & dine'}

In [49]:
topic_names_filepath = os.path.join(intermediate_directory, 'topic_names.pkl')

with open(topic_names_filepath, 'wb') as f:
    pickle.dump(topic_names, f)

You can see that, along with **mexican**, there are a variety of topics related to different styles of food, such as **thai**, **steak**, **sushi**, **pizza**, and so on. In addition, there are topics that are more related to the overall restaurant *experience*, like **ambience & seating**, **good service**, **waiting**, and **price**.

Beyond these two categories, there are still some topics that are difficult to apply a meaningful human interpretation to, such as topic 30 and 43.

Manually reviewing the top terms for each topic is a helpful exercise, but to get a deeper understanding of the topics and how they relate to each other, we need to visualize the data &mdash; preferably in an interactive format. Fortunately, we have the fantastic [**pyLDAvis**](https://pyldavis.readthedocs.io/en/latest/readme.html) library to help with that!

pyLDAvis includes a one-line function to take topic models created with gensim and prepare their data for visualization.

In [50]:
LDAvis_data_filepath = os.path.join(intermediate_directory, 'ldavis_prepared')

In [51]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 0:

    LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus,
                                              trigram_dictionary)

    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
        
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.77 ms


`pyLDAvis.display(...)` displays the topic model visualization in-line in the notebook.

In [52]:
pyLDAvis.display(LDAvis_prepared)

### Wait, what am I looking at again?
There are a lot of moving parts in the visualization. Here's a brief summary:

* On the left, there is a plot of the "distance" between all of the topics (labeled as the _Intertopic Distance Map_)
  * The plot is rendered in two dimensions according a [*multidimensional scaling (MDS)*](https://en.wikipedia.org/wiki/Multidimensional_scaling) algorithm. Topics that are generally similar should be appear close together on the plot, while *dis*similar topics should appear far apart.
  * The relative size of a topic's circle in the plot corresponds to the relative frequency of the topic in the corpus.
  * An individual topic may be selected for closer scrutiny by clicking on its circle, or entering its number in the "selected topic" box in the upper-left.
* On the right, there is a bar chart showing top terms.
  * When no topic is selected in the plot on the left, the bar chart shows the top-30 most "salient" terms in the corpus. A term's *saliency* is a measure of both how frequent the term is in the corpus and how "distinctive" it is in distinguishing between different topics.
  * When a particular topic is selected, the bar chart changes to show the top-30 most "relevant" terms for the selected topic. The relevance metric is controlled by the parameter $\lambda$, which can be adjusted with a slider above the bar chart.
    * Setting the $\lambda$ parameter close to 1.0 (the default) will rank the terms solely according to their probability within the topic.
    * Setting $\lambda$ close to 0.0 will rank the terms solely according to their "distinctiveness" or "exclusivity" within the topic &mdash; i.e., terms that occur *only* in this topic, and do not occur in other topics.
    * Setting $\lambda$ to values between 0.0 and 1.0 will result in an intermediate ranking, weighting term probability and exclusivity accordingly.
* Rolling the mouse over a term in the bar chart on the right will cause the topic circles to resize in the plot on the left, to show the strength of the relationship between the topics and the selected term.

A more detailed explanation of the pyLDAvis visualization can be found [here](https://cran.r-project.org/web/packages/LDAvis/vignettes/details.pdf). Unfortunately, though the data used by gensim and pyLDAvis are the same, they don't use the same ID numbers for topics. If you need to match up topics in gensim's `LdaMulticore` object and pyLDAvis' visualization, you have to dig through the terms manually.

### Analyzing our LDA model
The interactive visualization pyLDAvis produces is helpful for both:
1. Better understanding and interpreting individual topics, and
1. Better understanding the relationships between the topics.

For (1), you can manually select each topic to view its top most freqeuent and/or "relevant" terms, using different values of the $\lambda$ parameter. This can help when you're trying to assign a human interpretable name or "meaning" to each topic.

For (2), exploring the _Intertopic Distance Plot_ can help you learn about how topics relate to each other, including potential higher-level structure between groups of topics.

In our plot, there is a stark divide along the x-axis, with two topics far to the left and most of the remaining 48 far to the right. Inspecting the two outlier topics provides a plausible explanation: both topics contain many non-English words, while most of the rest of the topics are in English. So, one of the main attributes that distinguish the reviews in the dataset from one another is their language.

This finding isn't entirely a surprise. In addition to English-speaking cities, the Yelp dataset includes reviews of businesses in Montreal and Karlsruhe, Germany, often written in French and German, respectively. Multiple languages isn't a problem for our demo, but for a real NLP application, you might need to ensure that the text you're processing is written in English (or is at least tagged for language) before passing it along to some downstream processing. If that were the case, the divide along the x-axis in the topic plot would immediately alert you to a potential data quality issue.

The y-axis separates two large groups of topics &mdash; let's call them "super-topics" &mdash; one in the upper-right quadrant and the other in the lower-right quadrant. These super-topics correlate reasonably well with the pattern we'd noticed while naming the topics:
* The super-topic in the *lower*-right tends to be about *food*. It groups together the **burger & fries**, **breakfast**, **sushi**, **barbecue**, and **greek** topics, among others.
* The super-topic in the *upper*-right tends to be about other elements of the *restaurant experience*. It groups together the **ambience & seating**, **location & time**, **family**, and **customer service** topics, among others.

So, in addition to the 50 direct topics the model has learned, our analysis suggests a higher-level pattern in the data. Restaurant reviewers in the Yelp dataset talk about two main things in their reviews, in general: (1) the food, and (2) their overall restaurant experience. For this dataset, this is a very intuitive result, and we probably didn't need a sophisticated modeling technique to tell it to us. When working with datasets from other domains, though, such high-level patterns may be much less obvious from the outset &mdash; and that's where topic modeling can help.

### Describing text with LDA
Beyond data exploration, one of the key uses for an LDA model is providing a compact, quantitative description of natural language text. Once an LDA model has been trained, it can be used to represent free text as a mixture of the topics the model learned from the original corpus. This mixture can be interpreted as a probability distribution across the topics, so the LDA representation of a paragraph of text might look like 50% _Topic A_, 20% _Topic B_, 20% _Topic C_, and 10% _Topic D_.

To use an LDA model to generate a vector representation of new text, you'll need to apply any text preprocessing steps you used on the model's training corpus to the new text, too. For our model, the preprocessing steps we used include:
1. Using spaCy to remove punctuation and lemmatize the text
1. Applying our first-order phrase model to join word pairs
1. Applying our second-order phrase model to join longer phrases
1. Removing stopwords
1. Creating a bag-of-words representation

Once you've applied these preprocessing steps to the new text, it's ready to pass directly to the model to create an LDA representation. The `lda_description(...)` function will perform all these steps for us, including printing the resulting topical description of the input text.

In [53]:
def get_sample_review(review_number):
    """
    retrieve a particular review index
    from the reviews file and return it
    """
    
    return list(it.islice(line_review(review_txt_filepath),
                          review_number, review_number+1))[0]

In [54]:
def lda_description(review_text, min_topic_freq=0.05):
    """
    accept the original text of a review and (1) parse it with spaCy,
    (2) apply text pre-proccessing steps, (3) create a bag-of-words
    representation, (4) create an LDA representation, and
    (5) print a sorted list of the top topics in the LDA representation
    """
    
    # parse the review text with spaCy
    parsed_review = nlp(review_text)
    
    # lemmatize the text and remove punctuation and whitespace
    unigram_review = [token.lemma_ for token in parsed_review
                      if not punct_space(token)]
    
    # apply the first-order and secord-order phrase models
    bigram_review = bigram_model[unigram_review]
    trigram_review = trigram_model[bigram_review]
    
    # remove any remaining stopwords
    trigram_review = [term for term in trigram_review
                      if not term in spacy.en.STOPWORDS]
    
    # create a bag-of-words representation
    review_bow = trigram_dictionary.doc2bow(trigram_review)
    
    # create an LDA representation
    review_lda = lda[review_bow]
    # sort with the most highly related topics first
    review_lda = sorted(review_lda, key=lambda topic_freq: -topic_freq[1])
    
    for topic_number, freq in review_lda:
        if freq < min_topic_freq:
            break
            
        # print the most highly related topic names and frequencies
        print('{:25} {}'.format(topic_names[topic_number],
                                round(freq, 3)))

In [55]:
sample_review = get_sample_review(50)
print(sample_review)

Mediocre food. 50$ for one person. I could have had better food at Panda Express. The orange peel chicken was cold and lacking flavor. The veggies in the dish tasted like they weren't washed before being cooked. Only good thing about this place is the atmosphere, it is the only reason I didn't give it 1 star.



In [56]:
lda_description(sample_review)

healthy                   0.399
price                     0.245
sweet                     0.214
greek                     0.058


In [57]:
sample_review = get_sample_review(100)
print(sample_review)

Went here for dinner on a weekday and the live music was great, makes the experience much more enjoyable.  The service was good and not too over-the-top.

We loved the curry chicken, the garlic naan was good too, but I prefer regular naan.  Their spicy shrimp was very flavourful.  The quality of the dishes was very high.

The portions are much smaller than what I am used to at other Indian places though and it's a bit pricey too.  Overall, it was great though, I'd come back.



In [58]:
lda_description(sample_review)

greek                     0.449
wings, sports bar         0.219
price                     0.197
italian                   0.062


## Word Vector Embedding with Word2Vec

Pop quiz! Can you complete this text snippet?

<br><br>

![word2vec quiz](https://s3.amazonaws.com/skipgram-images/word2vec-1.png)

<br><br><br>
You just demonstrated the core machine learning concept behind word vector embedding models!
<br><br><br>

![word2vec quiz 2](https://s3.amazonaws.com/skipgram-images/word2vec-2.png)

The goal of *word vector embedding models*, or *word vector models* for short, is to learn dense, numerical vector representations for each term in a corpus vocabulary. If the model is successful, the vectors it learns about each term should encode some information about the *meaning* or *concept* the term represents, and the relationship between it and other terms in the vocabulary. Word vector models are also fully unsupervised &mdash they learn all of these meanings and relationships solely by analyzing the text of the corpus, without any advance knowledge provided.

Perhaps the best-known word vector model is [word2vec](https://arxiv.org/pdf/1301.3781v3.pdf), originally proposed in 2013. The general idea of word2vec is, for a given *focus word*, to use the *context* of the word &mdash; i.e., the other words immediately before and after it &mdash; to provide hints about what the focus word might mean. To do this, word2vec uses a *sliding window* technique, where it considers snippets of text only a few tokens long at a time.

At the start of the learning process, the model initializes random vectors for all terms in the corpus vocabulary. The model then slides the window across every snippet of text in the corpus, with each word taking turns as the focus word. Each time the model considers a new snippet, it tries to learn some information about the focus word based on the surrouding context, and it "nudges" the words' vector representations accordingly. One complete pass sliding the window across all of the corpus text is known as a training *epoch*. It's common to train a word2vec model for multiple passes/epochs over the corpus. Over time, the model rearranges the terms' vector representations such that terms that frequently appear in similar contexts have vector representations that are *close* to each other in vector space.

For a deeper dive into word2vec's machine learning process, see [here](https://arxiv.org/pdf/1411.2738v4.pdf).

Word2vec has a number of user-defined hyperparameters, including:
- The dimensionality of the vectors. Typical choices include a few dozen to several hundred.
- The width of the sliding window, in tokens. Five is a common default choice, but narrower and wider windows are possible.
- The number of training epochs.

For using word2vec in Python, [gensim](https://rare-technologies.com/deep-learning-with-word2vec-and-gensim/) comes to the rescue again! It offers a [highly-optimized](https://rare-technologies.com/word2vec-in-python-part-two-optimizing/), [parallelized](https://rare-technologies.com/parallelizing-word2vec-in-python/) implementation of the word2vec algorithm with its [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html) class.

In [59]:
from gensim.models import Word2Vec

trigram_sentences = LineSentence(trigram_sentences_filepath)
word2vec_filepath = os.path.join(intermediate_directory, 'word2vec_model_all')

We'll train our word2vec model using the normalized sentences with our phrase models applied. We'll use 100-dimensional vectors, and set up our training process to run for twelve epochs.

In [60]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the word2vec model yourself.
if 1 == 1:

    # initiate the model and perform the first epoch of training
    food2vec = Word2Vec(trigram_sentences, size=100, window=5,
                        min_count=20, sg=1, workers=4)
    
    food2vec.save(word2vec_filepath)

    # perform another 11 epochs of training
    for i in range(1,12):
        print('{} training epochs so far.'.format(food2vec.train_count))
        food2vec.train(trigram_sentences)
        food2vec.save(word2vec_filepath)
        
# load the finished model from disk
food2vec = Word2Vec.load(word2vec_filepath)
food2vec.init_sims()

print('{} training epochs so far.'.format(food2vec.train_count))

8 training epochs so far.
CPU times: user 1.28 s, sys: 124 ms, total: 1.41 s
Wall time: 1.4 s


On my four-core machine, each epoch over all the text in the ~1 million Yelp reviews takes about 5-10 minutes.

In [61]:
print('{:,} terms in the food2vec vocabulary.'.format(len(food2vec.vocab)))

90,320 terms in the food2vec vocabulary.


Let's take a peek at the word vectors our model has learned. We'll create a pandas DataFrame with the terms as the row labels, and the 100 dimensions of the word vector model as the columns.

In [63]:
# build a list of the terms, integer indices,
# and term counts from the food2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count)
                 for term, voc in food2vec.vocab.items()]

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda term_index_count:
                       -term_index_count[2])

# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# create a DataFrame with the food2vec vectors as data,
# and the terms as row labels
word_vectors = pd.DataFrame(food2vec.syn0norm[term_indices, :],
                            index=ordered_terms)

word_vectors

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
be,0.198263,0.030493,0.085409,0.050399,0.129970,-0.100306,-0.095698,0.265392,0.003230,0.027578,...,-0.032477,0.158966,-0.129142,0.030138,0.062846,0.102979,0.018772,-0.175671,0.005130,0.067239
the,0.126689,-0.097144,-0.023969,0.062254,0.204640,-0.109777,-0.006583,0.250808,0.041420,-0.055183,...,0.040028,0.014932,-0.069299,0.006080,0.058893,-0.042125,-0.063809,-0.063499,-0.018239,0.059399
and,0.098161,-0.025982,0.143218,-0.010929,0.098870,-0.180450,0.016660,0.063275,-0.040260,-0.023694,...,-0.011373,0.090081,-0.039313,0.004221,0.135217,-0.023702,0.059541,-0.142980,-0.017865,0.076215
i,0.036939,-0.049019,-0.040580,0.125575,0.138136,-0.101858,0.053984,0.019628,0.046230,0.062599,...,0.002715,0.117312,-0.106784,0.071469,0.285371,0.036882,0.080153,-0.040439,0.231806,0.168714
a,0.086840,0.050515,-0.084690,0.164698,0.152958,0.002217,0.031983,0.058809,0.043433,-0.084403,...,-0.032828,0.116924,-0.009765,0.018248,0.250621,0.052096,0.184647,-0.099691,0.062280,0.259210
to,0.028603,0.030927,-0.019692,-0.104867,0.095097,0.036237,0.032822,0.038548,-0.106778,-0.032828,...,0.034308,0.072256,-0.189388,-0.076984,0.068633,0.014882,0.131000,-0.175289,-0.014733,0.131706
it,-0.008247,0.108298,-0.005576,0.006867,0.123648,-0.101059,0.002321,0.004452,0.063237,-0.074009,...,-0.036650,0.113790,-0.246215,0.017902,0.200231,0.008449,0.283057,-0.051653,-0.031296,0.115539
have,0.085646,-0.026780,0.098442,0.092707,-0.018210,-0.099455,0.042673,0.231132,0.052572,-0.008235,...,0.058140,0.056907,-0.046760,0.053412,0.165893,-0.093730,0.034058,-0.129171,0.089945,-0.052885
of,0.044806,-0.069922,0.082944,0.047266,-0.002269,0.061813,0.014594,0.066156,-0.090468,-0.022137,...,-0.022753,0.216016,0.035395,-0.074023,0.131025,-0.004445,-0.004467,-0.021844,0.138124,0.048527
not,0.102250,0.142103,-0.046396,-0.082427,0.145607,-0.090905,0.084039,0.095011,-0.089820,0.064737,...,-0.129590,-0.017851,-0.082624,0.000477,0.106956,-0.064486,0.071990,0.018260,0.048885,-0.009778


Holy wall of numbers! This DataFrame has 50,835 rows &mdash; one for each term in the vocabulary &mdash; and 100 colums. Our model has learned a quantitative vector representation for each term, as expected.

Put another way, our model has "embedded" the terms into a 100-dimensional vector space.

### So... what can we do with all these numbers?
The first thing we can use them for is to simply look up related words and phrases for a given term of interest.

In [64]:
def get_related_terms(token, topn=10):
    """
    look up the topn most similar terms to token
    and print them as a formatted list
    """

    for word, similarity in food2vec.most_similar(positive=[token], topn=topn):

        print('{:20} {}'.format(word, round(similarity, 3)))

### What things are like Burger King?

In [65]:
get_related_terms(u'burger_king')

KeyError: "word 'burger_king' not in vocabulary"

### When is happy hour?

In [66]:
get_related_terms(u'happy_hour')

reverse_happy_hour   0.895
hh                   0.877
happy_hour-          0.837
happy_hr             0.828
during_happy_hour    0.807
hh.                  0.801
3_7pm                0.763
4_7pm                0.757
happy_hour_3_6pm     0.745
happy_hours          0.738


The model has noticed several alternate spellings for happy hour, such as *hh* and *happy hr*, and assesses them as highly related. If you were looking for reviews about happy hour, such alternate spellings would be very helpful to know.

Taking a deeper look &mdash; the model has turned up phrases like *3-6pm*, *4-7pm*, and *mon-fri*, too. This is especially interesting, because the model has no advance knowledge at all about what happy hour is, and what time of day it should be. But simply by scanning through restaurant reviews, the model has discovered that the concept of happy hour has something very important to do with that block of time around 3-7pm on weekdays.

### Let's make pasta tonight. Which style do you want?

In [67]:
get_related_terms(u'pasta', topn=20)

spaghetti            0.863
rigatoni             0.859
lasagna              0.855
penne                0.85
fettuccine           0.85
manicotti            0.847
bolognese            0.846
tortellini           0.839
lasagne              0.82
gnocchi              0.815
ravioli              0.813
fettucini            0.812
linguine             0.811
gnocci               0.81
linguini             0.808
penne_pasta          0.805
fettuccini           0.805
alfredo              0.804
fettucine            0.799
eggplant_parm        0.799


## Word algebra!
No self-respecting word2vec demo would be complete without a healthy dose of *word algebra*, also known as *analogy completion*.

The core idea is that once words are represented as numerical vectors, you can do math with them. The mathematical procedure goes like this:
1. Provide a set of words or phrases that you'd like to add or subtract.
1. Look up the vectors that represent those terms in the word vector model.
1. Add and subtract those vectors to produce a new, combined vector.
1. Look up the most similar vector(s) to this new, combined vector via cosine similarity.
1. Return the word(s) associated with the similar vector(s).

But more generally, you can think of the vectors that represent each word as encoding some information about the *meaning* or *concepts* of the word. What happens when you ask the model to combine the meaning and concepts of words in new ways? Let's see.

In [ ]:
def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = food2vec.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print term

### breakfast + lunch = ?
Let's start with a softball.

In [ ]:
word_algebra(add=[u'breakfast', u'lunch'])

OK, so the model knows that *brunch* is a combination of *breakfast* and *lunch*. What else?

### lunch - day + night = ?

In [ ]:
word_algebra(add=[u'lunch', u'night'], subtract=[u'day'])

Now we're getting a bit more nuanced. The model has discovered that:
- Both *lunch* and *dinner* are meals
- The main difference between them is time of day
- Day and night are times of day
- Lunch is associated with day, and dinner is associated with night

What else?

### taco - mexican + chinese = ?

In [ ]:
word_algebra(add=[u'taco', u'chinese'], subtract=[u'mexican'])

Here's an entirely new and different type of relationship that the model has learned.
- It knows that tacos are a characteristic example of Mexican food
- It knows that Mexican and Chinese are both styles of food
- If you subtract *Mexican* from *taco*, you're left with something like the concept of a *"characteristic type of food"*, which is represented as a new vector
- If you add that new *"characteristic type of food"* vector to Chinese, you get *dumpling*.

What else?

### bun - american + mexican = ?

In [ ]:
word_algebra(add=[u'bun', u'mexican'], subtract=[u'american'])

The model knows that both *buns* and *tortillas* are the doughy thing that goes on the outside of your real food, and that the primary difference between them is the style of food they're associated with.

What else?

### filet mignon - beef + seafood = ?

In [ ]:
word_algebra(add=[u'filet_mignon', u'seafood'], subtract=[u'beef'])

The model has learned a concept of *delicacy*. If you take filet mignon and subtract beef from it, you're left with a vector that roughly corresponds to delicacy. If you add the delicacy vector to *seafood*, you get *raw oyster*.

What else?

### coffee - drink + snack = ?

In [ ]:
word_algebra(add=[u'coffee', u'snack'], subtract=[u'drink'])

The model knows that if you're on your coffee break, but instead of drinking something, you're eating something... that thing is most likely a pastry.

What else?

### Burger King + fine dining = ?

In [ ]:
word_algebra(add=[u'burger_king', u'fine_dining'])

Touché. It makes sense, though. The model has learned that both Burger King and Denny's are large chains, and that both serve fast, casual, American-style food. But Denny's has some elements that are slightly more upscale, such as printed menus and table service. Fine dining, indeed.

*What if we keep going?*

### Denny's + fine dining = ?

In [ ]:
word_algebra(add=[u"denny_'s", u'fine_dining'])

This seems like a good place to land... what if we explore the vector space around *Applebee's* a bit, in a few different directions? Let's see what we find.

#### Applebee's + italian = ?

In [ ]:
word_algebra(add=[u"applebee_'s", u'italian'])

#### Applebee's + pancakes = ?

In [ ]:
word_algebra(add=[u"applebee_'s", u'pancakes'])

#### Applebee's + pizza = ?

In [ ]:
word_algebra(add=[u"applebee_'s", u'pizza'])

You could do this all day. One last analogy before we move on...

### wine - grapes + barley = ?

In [ ]:
word_algebra(add=[u'wine', u'barley'], subtract=[u'grapes'])

## Word Vector Visualization with t-SNE

[t-Distributed Stochastic Neighbor Embedding](https://lvdmaaten.github.io/publications/papers/JMLR_2008.pdf), or *t-SNE* for short, is a dimensionality reduction technique to assist with visualizing high-dimensional datasets. It attempts to map high-dimensional data onto a low two- or three-dimensional representation such that the relative distances between points are preserved as closely as possible in both high-dimensional and low-dimensional space.

scikit-learn provides a convenient implementation of the t-SNE algorithm with its [TSNE](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) class.

In [68]:
from sklearn.manifold import TSNE

Our input for t-SNE will be the DataFrame of word vectors we created before. Let's first:
1. Drop stopwords &mdash; it's probably not too interesting to visualize *the*, *of*, *or*, and so on
1. Take only the 5,000 most frequent terms in the vocabulary &mdash; no need to visualize all ~50,000 terms right now.

In [69]:
tsne_input = word_vectors.drop(spacy.en.STOPWORDS, errors='ignore')
tsne_input = tsne_input.head(5000)

In [70]:
tsne_input.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
food,0.070344,0.131230,-0.058306,-0.033444,0.121552,-0.117276,0.106413,-0.039369,-0.031554,-0.096803,...,0.198919,0.073903,-0.121361,0.020002,-0.019900,0.055775,0.073845,0.073190,0.142993,0.051441
place,0.037655,0.072575,-0.059044,0.126738,0.083559,0.010406,0.184988,0.055153,-0.222999,-0.194856,...,0.032005,0.231200,-0.232040,-0.029730,0.130613,-0.030447,0.078987,-0.049772,0.023560,0.019704
good,0.115156,-0.027908,0.101500,-0.164883,0.179799,-0.159016,0.060763,0.116665,0.041464,0.097663,...,-0.058423,-0.015670,-0.027894,0.180640,0.199613,-0.105352,0.123974,-0.054969,0.016556,0.029876
order,0.018773,-0.056044,0.160425,0.043430,0.013622,-0.054945,-0.054114,-0.067313,0.047801,0.181382,...,0.006881,0.015339,-0.011046,0.107479,0.120066,-0.034838,-0.044963,-0.082958,0.004243,0.127692
great,0.127893,-0.036812,0.042001,-0.170945,0.084564,-0.092603,0.005397,0.116946,0.084638,-0.057917,...,-0.017624,-0.027943,-0.031397,0.109228,0.213779,-0.003879,0.001928,-0.033440,-0.039307,0.005845


In [71]:
tsne_filepath = os.path.join(intermediate_directory,
                             'tsne_model')

tsne_vectors_filepath = os.path.join(intermediate_directory,
                                     'tsne_vectors.npy')

In [73]:
%%time

if 1 == 1:
    
    tsne = TSNE()
    tsne_vectors = tsne.fit_transform(tsne_input.values)
    
    with open(tsne_filepath, 'wb') as f:
        pickle.dump(tsne, f)

    pd.np.save(tsne_vectors_filepath, tsne_vectors)
    
with open(tsne_filepath, 'rb') as f:
    tsne = pickle.load(f)
    
tsne_vectors = pd.np.load(tsne_vectors_filepath)

tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(tsne_input.index),
                            columns=['x_coord', 'y_coord'])

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.34 ms


Now we have a two-dimensional representation of our data! Let's take a look.

In [74]:
tsne_vectors.head()

,x_coord,y_coord
food,7.590064,-2.394566
place,0.326041,-2.298657
good,6.721479,-6.715708
order,-5.355069,-8.873576
great,6.965602,-7.084775


In [75]:
tsne_vectors['word'] = tsne_vectors.index

### Plotting with Bokeh

In [76]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

output_notebook()

/home/fehiepsi/miniconda3/envs/pydata/lib/python3.5/site-packages/bokeh/core/json_encoder.py:52: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  NP_EPOCH = np.datetime64('1970-01-01T00:00:00Z')


Loading BokehJS ...

In [77]:
# add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(tsne_vectors)

# create the plot and configure the
# title, dimensions, and tools
tsne_plot = figure(title='t-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   tools= ('pan, wheel_zoom, box_zoom,'
                           'box_select, resize, reset'),
                   active_scroll='wheel_zoom')

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = '@word') )

# draw the words as circles on the plot
tsne_plot.circle('x_coord', 'y_coord', source=plot_data,
                 color='blue', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color='black')

# configure visual elements of the plot
tsne_plot.title.text_font_size = value('16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# engage!
show(tsne_plot);

## Conclusion

Whew! Let's round up the major components that we've seen:
1. Text processing with **spaCy**
1. Automated **phrase modeling**
1. Topic modeling with **LDA** $\ \longrightarrow\ $ visualization with **pyLDAvis**
1. Word vector modeling with **word2vec** $\ \longrightarrow\ $ visualization with **t-SNE**

#### Why use these models?
Dense vector representations for text like LDA and word2vec can greatly improve performance for a number of common, text-heavy problems like:
- Text classification
- Search
- Recommendations
- Question answering

...and more generally are a powerful way machines can help humans make sense of what's in a giant pile of text. They're also often useful as a pre-processing step for many other downstream machine learning applications.

## Data Science @ S&P Global &mdash; *we are hiring!*